In [30]:
import numpy as np
import pandas as pd

In [4]:
rggi = ['ME', 'NH', 'VT', 'MA', 'CT', 'RI', 'NY', 'NJ', 'DE', 'MD', 'VA']
rggi2 = ['NH', 'VT', 'MA', 'CT', 'RI', 'NY', 'NJ', 'DE', 'MD', 'VA']
california = ['CA']
other_states = ['AL', 'AK', 'AZ', 'AR', 'NC', 'SC', 'CO', 'ND', 'SD', 'FL', 'GA', 'HI', 'ID', 'IN', 'IL', 
                'IA', 'KS', 'KY', 'LA', 'MI', 'MN', 'MS', 'MT', 'MO', 'NE', 'NV', 'NM', 'OH', 'OK', 'OR', 'PA',
               'TN', 'TX', 'UT', 'WV', 'WA', 'WI', 'WY']
all_states = rggi + california + other_states
all_states2 = rggi2 + california + other_states

In [5]:
df_CO2CA_prices = pd.read_csv('treated_data/Cost/california_co2_prices.csv', sep = ';')
df_CO2RGGI_prices = pd.read_csv('treated_data/Cost/rggi_co2_prices.csv', sep = ';')
df_coal_prices = pd.read_csv('treated_data/Cost/coal_month_treated.csv', sep = ';')
df_gas_prices = pd.read_csv('treated_data/Cost/gaz_month_treated.csv', sep = ';')

In [72]:
df_panel = pd.read_csv('treated_data/Production/production_complete.csv', sep = ';')

In [5]:
df_panel.head()

,Unnamed: 0,Year,State,Plant_Id,Mover_Code,Fuel_Code,Month,Production(GWh),Technology
0,0,2006.0,AL,2.0,HY,WAT,1.0,12944.276,Hidroeletric
1,1,2006.0,AL,3.0,CA,NG,1.0,0.000,Natural Gas
2,2,2006.0,AL,3.0,CT,NG,1.0,81746.000,Natural Gas
3,3,2006.0,AL,3.0,ST,BIT,1.0,849774.000,Coal
4,4,2006.0,AL,3.0,ST,DFO,1.0,0.000,Petroleum Products


In [6]:
df_panel

,Unnamed: 0,Year,State,Plant_Id,Mover_Code,Fuel_Code,Month,Production(GWh),Technology
0,0,2006.0,AL,2.0,HY,WAT,1.0,12944.276,Hidroeletric
1,1,2006.0,AL,3.0,CA,NG,1.0,0.000,Natural Gas
2,2,2006.0,AL,3.0,CT,NG,1.0,81746.000,Natural Gas
3,3,2006.0,AL,3.0,ST,BIT,1.0,849774.000,Coal
4,4,2006.0,AL,3.0,ST,DFO,1.0,0.000,Petroleum Products
...,...,...,...,...,...,...,...,...,...
2117503,167503,2018.0,MA,99999.0,WT,WND,12.0,371.141,Wind
2117504,167504,2018.0,ND,99999.0,WT,WND,12.0,247.427,Wind
2117505,167505,2018.0,NY,99999.0,WT,WND,12.0,0.000,Wind
2117506,167506,2018.0,NY,99999.0,WT,WND,12.0,296.913,Wind


In [13]:
N = len(df_panel['Year'])
zero = np.zeros(N)
df_finalpanel = df_panel.assign(Coal_Cost=zero, Gas_Cost=zero, CO2_Cost=zero)  

In [14]:
def find_value_CO2CA(year, month):
    if month in [3, 6, 9, 12]:
        month-=1
    if month in [4, 7, 10]:
        month -=2
    if month == 1 :
        year -= 1
        month = 11
    if year < 2012:
        return(0)
    if year == 2012 and month < 11 :
        return(0)
    for element in df_CO2CA_prices['Unnamed: 0']:
        if df_CO2CA_prices['Year'][element] == year and df_CO2CA_prices['Month'][element] == month:
            return(df_CO2CA_prices.iloc[element, 4])

        
def find_value_CO2RGGI(year, month):
    if month in [4, 7, 10]:
        month -=1
    if month in [5, 8, 11]:
        month -=2
    if month == 1 or month == 2:
        month = 12
        year-=1    
    if year < 2008 :
        return(0)
    if year == 2008 and month < 12 :
        return(0)
    for element in df_CO2RGGI_prices['Unnamed: 0']:
        if df_CO2RGGI_prices['Year'][element] == year and df_CO2RGGI_prices['Month'][element] == month:
            return(df_CO2RGGI_prices.iloc[element, 4])

In [ ]:
for i in range (N) :
    element = i
    if df_finalpanel['State'][element] in rggi :
        year = df_finalpanel['Year'][element]
        month = df_finalpanel['Month'][element]
        co2_price = find_value_CO2RGGI(year, month)
        df_finalpanel['CO2_Cost'][element] = co2_price
    
    if df_finalpanel['State'][element] in california :
        year = df_finalpanel['Year'][element]
        month = df_finalpanel['Month'][element]
        co2_price = find_value_CO2CA(year, month)
        df_finalpanel['CO2_Cost'][element] = co2_price
        

In [17]:
df_finalpanel['Gas_Cost'][0]

0.0

In [19]:
def find_value_coal(year, month):
    index = year*100 + month
    for i in range(164):
        if df_coal_prices.iloc[i, 0] == index:
            return(df_coal_prices['Value'][i])
    return(0)

def find_value_gas(year, month):
    index = year*100 + month
    for i in range(164):
        if df_gas_prices.iloc[i, 0] == index:
            return(df_gas_prices['Value'][i])
    return(0)

In [20]:
find_value_coal(2013, 12)

43.41

In [ ]:
for element in range(N):
    year = df_finalpanel['Year'][element]
    month = df_finalpanel['Month'][element]
    coal_price = find_value_coal(year, month)
    gas_price = find_value_gas(year, month)
    df_finalpanel['Coal_Cost'][element] = coal_price
    df_finalpanel['Gas_Cost'][element] = gas_price
    

In [ ]:
df_finalpanel

In [65]:
df_finalpanel.to_excel('treated_data/Panel/panel.xlsx')

In [29]:
df_finalpanel.to_stata('treated_data/Panel/panel.dta')

In [164]:
df = pd.read_stata('treated_data/Panel/panel.dta')

# Écriture d'un df_CO2_prices complet de decemnre 2008 à juin 2021 sur tout les états 

In [ ]:
df_

In [20]:
df_CO2CA_prices = df_CO2CA_prices.assign(State='CA')  

In [21]:
df_join_CO2_RGGI = df_CO2RGGI_prices.assign(State = 'ME') 

In [22]:
for state in rggi2:
    df_prov = df_CO2RGGI_prices.assign(State = state)
    df_join_CO2_RGGI = df_join_CO2_RGGI.append(df_prov, ignore_index=True)

In [23]:
df_CO2_prices = df_join_CO2_RGGI.append(df_CO2CA_prices, ignore_index=True).drop(['Unnamed: 0','dates'], axis=1)

In [ ]:
n = len(df_CO2_prices)
for i in range(n):
    
    if df_CO2_prices['Month'][i] < 11 : 
        year = df_CO2_prices['Year'][i]
        month = df_CO2_prices['Month'][i]
        price = df_CO2_prices['CO2_prices'][i]
        state = df_CO2_prices['State'][i]
        df2 = pd.DataFrame([[year, month+1, price, state]], columns=['Year', 'Month', 'CO2_prices', 'State'])
        df3 = pd.DataFrame([[year, month+2, price, state]], columns=['Year', 'Month', 'CO2_prices', 'State'])
        if year < 2021 : 
            df_CO2_prices = df_CO2_prices.append(df2, ignore_index=True)
            df_CO2_prices = df_CO2_prices.append(df3, ignore_index=True)
        if year == 2021 :
            if month+1 < 7 :
                df_CO2_prices = df_CO2_prices.append(df2, ignore_index=True)
            if month +2 < 7 : 
                df_CO2_prices = df_CO2_prices.append(df3, ignore_index=True)
        
    if df_CO2_prices['Month'][i] == 11 : 
        year = df_CO2_prices['Year'][i]
        month = df_CO2_prices['Month'][i]
        price = df_CO2_prices['CO2_prices'][i]
        state = df_CO2_prices['State'][i]
        df2 = pd.DataFrame([[year, month+1, price, state]], columns=['Year', 'Month', 'CO2_prices', 'State'])
        df3 = pd.DataFrame([[year+1, 1, price, state]], columns=['Year', 'Month', 'CO2_prices', 'State'])
        df_CO2_prices = df_CO2_prices.append(df2, ignore_index=True)
        df_CO2_prices = df_CO2_prices.append(df3, ignore_index=True)
    
    if df_CO2_prices['Month'][i] == 12 : 
        year = df_CO2_prices['Year'][i]
        month = df_CO2_prices['Month'][i]
        price = df_CO2_prices['CO2_prices'][i]
        state = df_CO2_prices['State'][i]
        df2 = pd.DataFrame([[year+1, 1, price, state]], columns=['Year', 'Month', 'CO2_prices', 'State'])
        df3 = pd.DataFrame([[year+1, 2, price, state]], columns=['Year', 'Month', 'CO2_prices', 'State'])
        df_CO2_prices = df_CO2_prices.append(df2, ignore_index=True)
        df_CO2_prices = df_CO2_prices.append(df3, ignore_index=True)


In [25]:
for year in range(2008, 2013):
    for month in range(1, 13):
        if year == 2008:
            if month == 12 :
                df_price_null = pd.DataFrame([[year, month, 0, 'CA']],columns=['Year', 'Month', 'CO2_prices', 'State'])
                df_CO2_prices = df_CO2_prices.append(df_price_null, ignore_index=True)
        if year == 2012 : 
            if month < 11 :
                df_price_null = pd.DataFrame([[year, month, 0, 'CA']],columns=['Year', 'Month', 'CO2_prices', 'State'])
                df_CO2_prices = df_CO2_prices.append(df_price_null, ignore_index=True)
        if year >2008 and year < 2012 : 
            df_price_null = pd.DataFrame([[year, month, 0, 'CA']],columns=['Year', 'Month', 'CO2_prices', 'State'])
            df_CO2_prices = df_CO2_prices.append(df_price_null, ignore_index=True)
            

In [26]:
for year in range (2008, 2022):
    for month in range(1, 13):
        for state in other_states :
            
            if year > 2008 and year < 2021 :
                df_price_null = pd.DataFrame([[year, month, 0, state]],columns=['Year', 'Month', 'CO2_prices', 'State'])
                df_CO2_prices = df_CO2_prices.append(df_price_null, ignore_index=True)
            
            if year == 2008 : 
                if month == 12 : 
                    df_price_null = pd.DataFrame([[year, month, 0, state]],columns=['Year', 'Month', 'CO2_prices', 'State'])
                    df_CO2_prices = df_CO2_prices.append(df_price_null, ignore_index=True)
                    
            if year == 2021 : 
                if month < 7 : 
                    df_price_null = pd.DataFrame([[year, month, 0, state]],columns=['Year', 'Month', 'CO2_prices', 'State'])
                    df_CO2_prices = df_CO2_prices.append(df_price_null, ignore_index=True)

In [27]:
df_CO2_prices

,Year,Month,C02 prices,State,CO2_prices
0,2008,12,3.725812,ME,NaN
1,2009,3,3.869113,ME,NaN
2,2009,6,3.560466,ME,NaN
3,2009,9,2.061322,ME,NaN
4,2009,12,2.050299,ME,NaN
...,...,...,...,...,...
6376,2021,6,NaN,UT,0.0
6377,2021,6,NaN,WV,0.0
6378,2021,6,NaN,WA,0.0
6379,2021,6,NaN,WI,0.0


# Ecriture d'un df_coal_gas prices par état 

In [18]:
df_gas_prices_clear = df_gas_prices.drop(['YYYYMM', 'Unit', 'Production'], axis=1)[:164]

In [19]:
df_coal_prices_clear = df_coal_prices.drop(['YYYYMM', 'Unit', 'Production'], axis=1)

In [387]:
df_coal_prices_clear = df_coal_prices_clear.rename(columns = {'Value' : 'Coal_prices'})
df_gas_prices_clear = df_gas_prices_clear.rename(columns = {'Value' : 'Gas_prices'})

In [388]:
df_coal_gas_prices = pd.merge(df_coal_prices_clear, df_gas_prices_clear, on=['Year', 'Month'])

In [389]:
df_coal_gas_prices_final = df_coal_gas_prices.assign(State = 'ME')

In [390]:
for state in all_states2:
    df_prov = df_coal_gas_prices.assign(State = state)
    df_coal_gas_prices_final = df_coal_gas_prices_final.append(df_prov, ignore_index=True)

In [392]:
df_coal_gas_prices_final

,Coal_prices,Year,Month,Gas_prices,State
0,37.06,2007,12,8.3175,ME
1,37.18,2008,1,8.6599,ME
2,38.27,2008,2,9.2982,ME
3,38.86,2008,3,9.9365,ME
4,40.35,2008,4,10.6947,ME
...,...,...,...,...,...
8195,36.26,2021,3,3.0374,WY
8196,36.07,2021,4,3.2478,WY
8197,37.40,2021,5,3.4601,WY
8198,37.79,2021,6,3.9907,WY


In [393]:
df_coal_gas_prices_final.where(df_coal_gas_prices_final.State == 'CA').dropna()

,Coal_prices,Year,Month,Gas_prices,State
1804,37.06,2007.0,12.0,8.3175,CA
1805,37.18,2008.0,1.0,8.6599,CA
1806,38.27,2008.0,2.0,9.2982,CA
1807,38.86,2008.0,3.0,9.9365,CA
1808,40.35,2008.0,4.0,10.6947,CA
...,...,...,...,...,...
1963,36.26,2021.0,3.0,3.0374,CA
1964,36.07,2021.0,4.0,3.2478,CA
1965,37.40,2021.0,5.0,3.4601,CA
1966,37.79,2021.0,6.0,3.9907,CA


In [394]:
df_prices = pd.merge(df_coal_gas_prices_final, df_CO2_prices, on=['Year', 'Month', 'State'])

In [396]:
df_prices = df_prices[['Year', 'Month', 'State', 'Coal_prices', 'Gas_prices', 'CO2_prices']]

In [439]:
df_prices

,Year,Month,State,Coal_prices,Gas_prices,CO2_prices
0,2008,12,ME,43.30,6.4407,3.725812
1,2009,1,ME,44.57,5.4807,3.725812
2,2009,2,ME,45.57,4.7999,3.725812
3,2009,3,ME,43.96,4.5772,3.869113
4,2009,4,ME,44.02,4.5832,3.869113
...,...,...,...,...,...,...
7545,2021,2,WY,36.84,3.2943,0.000000
7546,2021,3,WY,36.26,3.0374,0.000000
7547,2021,4,WY,36.07,3.2478,0.000000
7548,2021,5,WY,37.40,3.4601,0.000000


# Merge général

In [463]:
df_production = pd.read_csv('treated_data/Panel/panel_ii.csv', sep=';')

In [464]:
df_production = df_production.drop(['Unnamed: 0'], axis=1)
df_production

,Year,Month,Plant_Id,State,Mover_Code,Fuel_Code,Production(GWh),Demand,Wind_Production,Solar_Production
0,2006,1.0,3.0,AL,ST,BIT,849774.0,3.286577e+08,0.000,0.000
1,2006,1.0,7.0,AL,ST,BIT,31219.246,3.286577e+08,0.000,0.000
2,2006,1.0,8.0,AL,ST,BIT,602299.0,3.286577e+08,0.000,0.000
3,2006,1.0,10.0,AL,ST,BIT,341354.0,3.286577e+08,0.000,0.000
4,2006,1.0,26.0,AL,ST,BIT,956562.0,3.286577e+08,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...
139580,2018,12.0,99999.0,KS,ST,SUB,0,3.421391e+08,1795033.304,198.555
139581,2018,12.0,99999.0,MO,ST,SUB,.,3.421391e+08,267592.037,4609.609
139582,2018,12.0,99999.0,OR,ST,SUB,.,3.421391e+08,388313.064,26048.597
139583,2018,12.0,99999.0,TN,ST,SUB,0,3.421391e+08,3565.316,9062.800


In [465]:
df_prices_join = df_prices.groupby(['Year', 'Month', 'State']).sum()

In [466]:
df_final = df_production.join(df_prices_join, on=['Year', 'Month', 'State'], how = 'left')

In [472]:
panel = df_final.dropna()

In [482]:
len(panel.Plant_Id.unique())

772

In [473]:
panel

,Year,Month,Plant_Id,State,Mover_Code,Fuel_Code,Production(GWh),Demand,Wind_Production,Solar_Production,Coal_prices,Gas_prices,CO2_prices
18296,2010,1.0,3.0,AL,ST,BIT,703950.78,3.609572e+08,0.000,0.000,44.37,6.1428,0.0
18297,2010,1.0,7.0,AL,ST,BIT,17062.656,3.609572e+08,0.000,0.000,44.37,6.1428,0.0
18298,2010,1.0,8.0,AL,ST,BIT,702499.107,3.609572e+08,0.000,0.000,44.37,6.1428,0.0
18299,2010,1.0,10.0,AL,ST,BIT,300108.234,3.609572e+08,0.000,0.000,44.37,6.1428,0.0
18300,2010,1.0,26.0,AL,ST,BIT,779059.632,3.609572e+08,0.000,0.000,44.37,6.1428,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139580,2018,12.0,99999.0,KS,ST,SUB,0,3.421391e+08,1795033.304,198.555,40.03,4.0210,0.0
139581,2018,12.0,99999.0,MO,ST,SUB,.,3.421391e+08,267592.037,4609.609,40.03,4.0210,0.0
139582,2018,12.0,99999.0,OR,ST,SUB,.,3.421391e+08,388313.064,26048.597,40.03,4.0210,0.0
139583,2018,12.0,99999.0,TN,ST,SUB,0,3.421391e+08,3565.316,9062.800,40.03,4.0210,0.0


In [485]:
panel.to_excel('treated_data/Panel/panel_final.xlsx')